In [ ]:
!pip -q install geopandas rasterio rasterstats shapely

     |████████████████████████████████| 1.0 MB 31.6 MB/s 
     |████████████████████████████████| 19.3 MB 6.8 MB/s 
     |████████████████████████████████| 6.3 MB 54.7 MB/s 
     |████████████████████████████████| 16.7 MB 56.2 MB/s 
     |████████████████████████████████| 130 kB 76.2 MB/s 


In [ ]:
from shapely.geometry import Point
from math import sqrt
import cv2
from rasterio.features import rasterize
import numpy as np
import rasterio
from rasterio import plot
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
import geopandas

def to_square(polygon):
    minx, miny, maxx, maxy = polygon.bounds
    centroid = [(maxx+minx)/2, (maxy+miny)/2]
    return Point(centroid).buffer(84, cap_style=3)

In [ ]:
path_data_list = glob('/content/drive/MyDrive/Varuna Hackathon 2022/sentinel-2-image/2021/2021*/IMG_DATA/***.jp2')
img_profile = rasterio.open('/content/drive/MyDrive/Varuna Hackathon 2022/sentinel-2-image/2021/20211227/IMG_DATA/47PQS_20211227_B11.jp2').profile
label = geopandas.read_file('/content/drive/MyDrive/Varuna Hackathon 2022/training_area/traindata.shp').to_crs(img_profile['crs'])

arr_all = []
y = []
for idx, path_data in tqdm(enumerate([path_data_list[0]])): # if you want to full prepare data Please enter 'path_data_list' instead of '[path_data_list[0]]'
    print(idx)
    img_for_crop = rasterio.open(path_data).read(1)
    img_profile = rasterio.open(path_data).profile
    arr_day = []
    for i, row in label.iterrows():
        if row.crop_type in ['1', '2', '3', '4']:
            geometry = to_square(row.geometry)
            crop_type =  int(row.crop_type)
            target_image = rasterize(shapes=[geometry, crop_type],
                                    out_shape=(img_profile['width'], img_profile['height']),
                                    transform=img_profile['transform'])
            img = target_image.astype(np.uint8)
            x_coords = np.where((img > 0))[0]
            y_coords = np.where((img > 0))[1]
            xmin = int(min(x_coords))
            ymin = int(max(y_coords))
            w = int(max(x_coords) - min(x_coords))
            h = int(max(y_coords) - min(y_coords))
            result_img = img_for_crop[ymin:ymin+h, xmin:xmin+w]
            result_img = cv2.resize(result_img, (16, 16))
            each_band = result_img
            arr_day.append([each_band])
            if idx < 1:
                y.append(crop_type)
    arr_all.append(arr_day)

0it [00:00, ?it/s]

0


/usr/local/lib/python3.7/dist-packages/rasterio/features.py:318: ShapeSkipWarning: Invalid or empty shape 1 at index 1 will not be rasterized.
  warnings.warn('Invalid or empty shape {} at index {} will not be rasterized.'.format(geom, index), ShapeSkipWarning)
/usr/local/lib/python3.7/dist-packages/rasterio/features.py:318: ShapeSkipWarning: Invalid or empty shape 3 at index 1 will not be rasterized.
  warnings.warn('Invalid or empty shape {} at index {} will not be rasterized.'.format(geom, index), ShapeSkipWarning)
/usr/local/lib/python3.7/dist-packages/rasterio/features.py:318: ShapeSkipWarning: Invalid or empty shape 2 at index 1 will not be rasterized.
  warnings.warn('Invalid or empty shape {} at index {} will not be rasterized.'.format(geom, index), ShapeSkipWarning)
/usr/local/lib/python3.7/dist-packages/rasterio/features.py:318: ShapeSkipWarning: Invalid or empty shape 4 at index 1 will not be rasterized.
  warnings.warn('Invalid or empty shape {} at index {} will not be rast

In [ ]:
arr_all = np.array(arr_all)
y = np.array(y)

arr_all.shape, y.shape #arr_all is 5-dimension for CNN (N-band, N-crop, day, w, h)

((1, 1317, 1, 16, 16), (1317,))

In [ ]:
with open('/content/drive/MyDrive/ARV_hackthon_2022/arr_all.npy', 'wb') as aa:
    np.save(aa, arr_all)
with open('/content/drive/MyDrive/ARV_hackthon_2022/labels.npy', 'wb') as l:
    np.save(l, y)